# Convolutional Neural Networks for Human Activity Recognition
***
# Table of Contents

# Imports

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.nn import Linear, Conv1d, MaxPool1d, Module, CrossEntropyLoss, Dropout, ReLU, Flatten, Sequential, Softmax
from torch.optim import Adam
from torch.utils.data import Dataset
import torch.nn.functional as F

# Encoding Map

In [ ]:
dynamic_encode =    {
                        "WALKING":0,
                        "WALKING_DOWNSTAIRS":1,
                        "WALKING_UPSTAIRS":2,
                        "LAYING":3,
                        "SITTING":4,
                        "STANDING":5
                    }

static_encode =     {
                        "LAYING":0,
                        "SITTING":1,
                        "STANDING":2,
                        "WALKING":3,
                        "WALKING_DOWNSTAIRS":4,
                        "WALKING_UPSTAIRS":5
                    }

# Creating PyTorch Datasets

## Static Dataset

In [ ]:
class UCI_Static_Dataset(Dataset):
    """UCI dataset."""

    def __init__(self, csv_file):

        data = pd.read_csv(csv_file)
        data['Activity'] = data['Activity'].map(static_encode)
        data, _ = [x for _, x in data.groupby(data['Activity'] > 2)]
        self.data_y = data['Activity'].values
        data = pd.DataFrame(data.drop(['Activity','subject'],axis=1))
        self.data_x = np.array(data)
        # [batch, channels, features]
        self.data_x = self.data_x.reshape(len(self.data_x), 1, 561)
        self.data_x  = torch.from_numpy(self.data_x)
        self.data_y = self.data_y.astype(int)
        self.data_y = torch.from_numpy(self.data_y)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]

## Dynamic Dataset

In [ ]:
class UCI_Dynamic_Dataset(Dataset):
    """UCI dataset."""

    def __init__(self, csv_file):
        data = pd.read_csv(csv_file)
        data['Activity'] = data['Activity'].map(dynamic_encode)
        data, _ = [x for _, x in data.groupby(data['Activity'] > 2)]
        self.data_y = data['Activity'].values
        data = pd.DataFrame(data.drop(['Activity','subject'],axis=1))
        self.data_x = np.array(data)
        # [batch, channels, features]
        self.data_x = self.data_x.reshape(len(self.data_x), 1, 561)
        self.data_x  = torch.from_numpy(self.data_x)
        self.data_y = self.data_y.astype(int)
        self.data_y = torch.from_numpy(self.data_y)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]


# Loading in the Data

In [ ]:
data = {
        "dynamic":  {
                    "train":    UCI_Dynamic_Dataset(csv_file='train.csv'),
                    "valid:":   UCI_Dynamic_Dataset(csv_file='valid.csv'),
                    "test:":    UCI_Dynamic_Dataset(csv_file='test.csv')
                    } ,
        "static":   {
                    "train":    UCI_Static_Dataset(csv_file='train.csv'),
                    "valid:":   UCI_Static_Dataset(csv_file='valid.csv'),
                    "test:":    UCI_Static_Dataset(csv_file='test.csv')
                    }
        }


# Trainloaders

In [ ]:
loader = {
        "dynamic":  {
                    "train":    torch.utils.data.DataLoader(data["dynamic"]["train"], batch_size=64, shuffle=True),
                    "valid:":   torch.utils.data.DataLoader(data["dynamic"]["valid"], batch_size=64, shuffle=True),
                    "test:":    torch.utils.data.DataLoader(data["dynamic"]["test"], batch_size=64, shuffle=True)
                    } ,
        "static":   {
                    "train":    torch.utils.data.DataLoader(data["static"]["train"], batch_size=32, shuffle=True),
                    "valid:":   torch.utils.data.DataLoader(data["static"]["valid"], batch_size=32, shuffle=True),
                    "test:":    torch.utils.data.DataLoader(data["static"]["test"], batch_size=32, shuffle=True)
                    }
        }

# The CNN Models

# Static

In [ ]:
class StaticCNN(Module):
    def __init__(self):
        super(StaticCNN, self).__init__()

        self.cnn_layers = Sequential(
            Conv1d(1, 30, kernel_size=tuple([3])),
            ReLU(),
            Conv1d(30, 50, kernel_size=tuple([3])),
            ReLU(),
            Conv1d(50, 100, kernel_size=tuple([3])),
            ReLU(),
            Flatten()
        )

        self.linear_layers = Sequential(
            Dropout(0.5),
            Linear(55500, 3),
            Softmax(dim=1)
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.linear_layers(x)
        return x

# Dynamic

In [ ]:
class DynamicCNN(Module):
    def __init__(self):
        super(DynamicCNN, self).__init__()

        self.conv1 = Conv1d(1, 100, kernel_size=tuple([3]))
        self.pool =  MaxPool1d(kernel_size=tuple([3]))
        self.fc = Linear(18600, 3)
        self.dropout = Dropout(0.5)

    # Defining the forward pass
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 1 * 100 * 186)
        x = self.dropout(F.softmax(self.fc(x), dim=1))
        return x